In [88]:
import requests 
import csv
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re 
pd.set_option('display.max_colwidth', 1000)

In [ ]:
#Getting the HTML and isolating the section we need 
tse_parties = requests.get("http://www.tse.jus.br/partidos/partidos-politicos/registrados-no-tse")
tse_soup = BeautifulSoup(tse_parties.content, 'html.parser')
tse_soup_content = tse_soup.find_all(class_='documentContent')[0]#.get_text()
parties = tse_soup.find("caption", text="Partidos registrados no TSE").find_parent("table")

In [27]:
#Interpreting the data to save in a .csv
headers = ""
for val in parties.find_all('th'):
    headers += val.text + ','
headers = headers[:-1]

rows = []
for row in parties.find_all('tr'):
    row1 = ""
    for val in row.find_all('td'):
        row1 += val.text + ','
    row1 = row1[:-1]
    rows.append(row1)
    
#Removing last line
rows = rows[:-1]

with open('parties.csv', 'w') as f:
    writer = csv.writer(f)
    #writer = csv.writer(f)
    header = (headers).encode("utf-8")
    writer.writerow([header])
    for row in rows:
        outputrow = (row).encode("utf-8")
        writer.writerow([outputrow])#(row)

#TODO: clean quotes - currently being done manually

In [30]:
#Saving data intro a dataframe
partiescsv = pd.read_csv('parties.csv', index_col=False, header=0);

#Adding TSE Url to dataframe
partiescsv['tseurl'] = ''
siglas = partiescsv['SIGLA']

for website_found in parties.find_all('a', class_='internal-link'):
    index = siglas[siglas == website_found.text].index
    partiescsv.set_value(index, 'tseurl', str(website_found['href']))
    
partiescsv.head()
#print partiescsv

#TODO: 
#1. TSE url truncating
#2. 'no exercicio da presidencia' cleaned manually




,Unnamed: 0,0001,SIGLA,NOME,DEFERIMENTO,PRES. NACIONAL,Nº DA LEGENDA,tseurl
0,0,1,PMDB,PARTIDO DO MOVIMENTO DEMOCRÁTICO BRASILEIRO,30.6.1981,ROMERO JUCÁ,15,http://www.tse.jus.br/partidos/partidos-politicos/partido-do-movimento-democratico-brasileiro
1,1,2,PTB,PARTIDO TRABALHISTA BRASILEIRO,3.11.1981,ROBERTO JEFFERSON MONTEIRO FRANCISCO,14,http://www.tse.jus.br/partidos/partidos-politicos/partido-trabalhista-brasileiro
2,2,3,PDT,PARTIDO DEMOCRÁTICO TRABALHISTA,10.11.1981,CARLOS LUPI,12,http://www.tse.jus.br/partidos/partidos-politicos/partido-democratico-trabalhista
3,3,4,PT,PARTIDO DOS TRABALHADORES,11.2.1982,RUI GOETHE DA COSTA FALCAO,13,http://www.tse.jus.br/partidos/partidos-politicos/partido-dos-trabalhadores
4,4,5,DEM,DEMOCRATAS,11.9.1986,JOSÉ AGRIPINO MAIA,25,http://www.tse.jus.br/partidos/partidos-politicos/democratas


In [231]:
#Fetching further party information from TSE's website
tse_party_url = partiescsv['tseurl'][0]

urls = partiescsv['tseurl']

#Adding Party fields to dataframe
partiescsv['telefones'] = ''
partiescsv['fax'] = ''
partiescsv['cep'] = ''
partiescsv['email'] = ''

for tseurl in partiescsv['tseurl']:
    index = urls[urls == tseurl].index
    
    #Getting the HTML and isolating the section we need 
    party_soup = BeautifulSoup(requests.get(tseurl).content, 'html.parser')
#    party_soup_content = pmdb_soup.find_all(class_='documentContent')[0]#.get_text()
    party_infos = party_soup.find_all("h3", text="Informações sobre o partido")[0].parent#.find_parent("table")

    #print(pmdb_infos)
    phones = party_infos.find_all("p", text=re.compile('Telefone'))
    phones = phones[0].text.split(':')[1] if len(phones) > 0 else 'null'
    fax = party_infos.find_all("p", text=re.compile('FAX:'))
    fax = fax[0].text.split(':')[1] if len(fax) > 0 else 'null'
    cep = party_infos.find_all("p", text=re.compile('CEP:'))
    cep = cep[0].text.split(':')[1] if len(cep) > 0 else 'null'
    emails = [a.text for a in party_infos.select('a[href^=mailto:]')]
    
    partiescsv.set_value(index, 'telefones', str(phones))
    partiescsv.set_value(index, 'fax', str(fax))
    partiescsv.set_value(index, 'cep', str(cep))
    partiescsv.set_value(index, 'email', str(emails))



#TODO:
#1. Improve email detection
#2. Implement party website detection
#4. Drop initial trash columns 

In [227]:
partiescsv.head()

,Unnamed: 0,0001,SIGLA,NOME,DEFERIMENTO,PRES. NACIONAL,Nº DA LEGENDA,tseurl,telefones,fax,cep,email
0,0,1,PMDB,PARTIDO DO MOVIMENTO DEMOCRÁTICO BRASILEIRO,30.6.1981,ROMERO JUCÁ,15,http://www.tse.jus.br/partidos/partidos-politicos/partido-do-movimento-democratico-brasileiro,"Telefones: (61) 2192 9171, (61) 2192 9365",FAX: (61) 2192 9336,CEP: 71.630-275,[u'diretorionacional.pmdb@uol.com.br']
1,1,2,PTB,PARTIDO TRABALHISTA BRASILEIRO,3.11.1981,ROBERTO JEFFERSON MONTEIRO FRANCISCO,14,http://www.tse.jus.br/partidos/partidos-politicos/partido-trabalhista-brasileiro,Telefone: (61) 2101.1414,FAX: (61) 2101 1400,CEP: 70730-521,"[u'ptb@ptb.org.br', u'juridico@ptb.org.br', u'luizgustavo@ptb.org.br']"
2,2,3,PDT,PARTIDO DEMOCRÁTICO TRABALHISTA,10.11.1981,CARLOS LUPI,12,http://www.tse.jus.br/partidos/partidos-politicos/partido-democratico-trabalhista,Telefone: (61) 3224.0791,FAX: (61) 3225.4290,CEP: 70042-900,[u'secretarianacional@pdt.org.br']
3,3,4,PT,PARTIDO DOS TRABALHADORES,11.2.1982,RUI GOETHE DA COSTA FALCAO,13,http://www.tse.jus.br/partidos/partidos-politicos/partido-dos-trabalhadores,Telefone: (61) 3213.1313,FAX: (61) 3213.1313,CEP: 70302-000,[u'presidencia@pt.org.br']
4,4,5,DEM,DEMOCRATAS,11.9.1986,JOSÉ AGRIPINO MAIA,25,http://www.tse.jus.br/partidos/partidos-politicos/democratas,Telefone: (61) 3311-4305,FAX: (61) 3224-1912,CEP: 70165-900,[u'democratas25@democratas.org.br']


In [241]:
#Saving computed data out 

#partiescsv.drop('0001', axis=1, inplace=True)
#partiescsv.drop('Unnamed: 0', axis=1, inplace=True)
partiescsv.to_csv('parties1.csv', encoding='utf-8')